In [3]:
import pandas as pd
import ast
from sklearn.metrics import precision_score, recall_score, f1_score

In [4]:
def safe_literal_eval(cell_str):
    try:
        return ast.literal_eval(cell_str)
    except (SyntaxError, ValueError, TypeError):
        return None

In [5]:
def compute_model_correct_percentages(df, solution_col):
    metadata_columns = {"tokens", "ID", "ner_tags", "collection", "Source"}
    predict_columns = [col for col in df.columns if col not in metadata_columns]
    
    results = {}
    total_rows = len(df)
    
    for col in predict_columns:
        mismatch_count = 0
        skipped_count = 0
        
        for _, row in df.iterrows():
            list_solution = safe_literal_eval(row[solution_col])
            list_pred = safe_literal_eval(row[col])
            
            if list_solution is None or list_pred is None:
                skipped_count += 1
                continue
            
            if len(list_solution) != len(list_pred):
                mismatch_count += 1
        
        usable_rows = total_rows - skipped_count

        if usable_rows == 0:
            correct_percentage = None  
        else:
            correct_percentage = ((usable_rows - mismatch_count) / usable_rows) * 100
        
        results[col] = correct_percentage
    
    return results

datasets = [
    ("Dataset1", "PET-Customized-1.csv"),
    ("Dataset2", "PET-Customized-2.csv"),
    ("Dataset3", "PET-Customized-3.csv")
]

dataset_results = {}

In [6]:
for name, path in datasets:
    df = pd.read_csv(path)
    dataset_results[name] = compute_model_correct_percentages(df, solution_col="ner_tags")

all_models = set()
for result in dataset_results.values():
    all_models.update(result.keys())

table_rows = []

In [7]:
for model in sorted(all_models):
    row_data = {"Model": model}
    values = []  
    for dataset_name, _ in datasets:
        value = dataset_results[dataset_name].get(model)
        row_data[dataset_name] = value
        if value is not None:
            values.append(value)
    row_data["Average"] = sum(values) / len(values) if values else None
    table_rows.append(row_data)

In [8]:
results_df = pd.DataFrame(table_rows).set_index("Model")
print(results_df)

                              Dataset1   Dataset2   Dataset3    Average
Model                                                                  
NER_Tag_Claude3.0-Haiku       0.000000   0.000000   0.000000   0.000000
NER_Tag_Claude3.5-Haiku       1.438849   1.438849   1.678657   1.518785
NER_Tag_Claude3.5-Sonnet      6.714628   6.235012   5.995204   6.314948
NER_Tag_DeepSeek_Chat        11.195929   9.832134  11.309524  10.779196
NER_Tag_GPT4o                15.587530  12.709832  14.148681  14.148681
NER_Tag_GPT4o-mini            0.000000   0.000000   0.000000   0.000000
NER_Tag_Grok2                 9.832134   9.832134   7.913669   9.192646
NER_Tag_Llama-3.1-70B         3.597122   4.556355   3.357314   3.836930
NER_Tag_Llama-3.1-8B          0.000000   0.239808   0.000000   0.079936
NER_Tag_gemini-1.0-pro        3.836930   2.637890   3.357314   3.277378
NER_Tag_gemini-1.5-flash      0.000000   0.000000   0.000000   0.000000
NER_Tag_gemini-1.5-flash-8b   1.476015   1.115242   1.945525   1

In [9]:
def calculate_metrics(df, solution_col):
    relevant_tags = {'B-Activity Data', 'I-Activity Data', 'B-Activity', 
                     'I-Activity', 'B-Actor', 'I-Actor'}
    
    metadata_columns = {"tokens", "ID", "ner_tags", "collection", "source"}
    predict_columns = [col for col in df.columns if col not in metadata_columns]

    results = {}
    for col in predict_columns:
        y_true, y_pred = [], []
        for _, row in df.iterrows():
            solution_list = safe_literal_eval(row[solution_col])
            predicted_list = safe_literal_eval(row[col])
            
            if solution_list is None or predicted_list is None:
                continue
            
            for sol_tag, pred_tag in zip(solution_list, predicted_list):
                if sol_tag in relevant_tags:
                    y_true.append(sol_tag)
                    y_pred.append(pred_tag)

        if not y_true or not y_pred:
            continue

        precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
        recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
        f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
        results[col] = {"Precision": precision, "Recall": recall, "F1-Score": f1}

    return results

In [10]:
datasets = [
    ("Dataset1", "PET-Customized-1.csv"),
    ("Dataset2", "PET-Customized-2.csv"),
    ("Dataset3", "PET-Customized-3.csv")
]

dataset_metrics = {}

for ds_name, ds_path in datasets:
    df = pd.read_csv(ds_path)
    dataset_metrics[ds_name] = calculate_metrics(df, solution_col="ner_tags")

all_models = set()
for ds in dataset_metrics.values():
    all_models.update(ds.keys())

final_results = {}
for metric in ["Precision", "Recall", "F1-Score"]:
    table = []
    for model in sorted(all_models):
        row = {"Model": model}
        values = []
        for ds_name, _ in datasets:
            ds_result = dataset_metrics.get(ds_name, {}).get(model)
            value = ds_result.get(metric) if ds_result is not None else None
            row[ds_name] = value
            if value is not None:
                values.append(value)
        row["Average"] = sum(values) / len(values) if values else None
        table.append(row)
    final_results[metric] = table

for metric, table in final_results.items():
    print(f"\n{metric} results:")
    df_metric = pd.DataFrame(table).set_index("Model")
    print(df_metric)


Precision results:
                             Dataset1  Dataset2  Dataset3   Average
Model                                                              
NER_Tag_Claude3.0-Haiku      0.116000  0.116000  0.117161  0.116387
NER_Tag_Claude3.5-Haiku      0.183086  0.184022  0.202477  0.189862
NER_Tag_Claude3.5-Sonnet     0.249724  0.227706  0.231510  0.236313
NER_Tag_DeepSeek_Chat        0.286927  0.215285  0.164842  0.222351
NER_Tag_GPT4o                0.180466  0.183566  0.218980  0.194337
NER_Tag_GPT4o-mini           0.128459  0.166631  0.165064  0.153385
NER_Tag_Grok2                0.267489  0.289038  0.255529  0.270686
NER_Tag_Llama-3.1-70B        0.141092  0.152773  0.112161  0.135342
NER_Tag_Llama-3.1-8B         0.032857  0.038429  0.031837  0.034374
NER_Tag_gemini-1.0-pro       0.059974  0.037779  0.027565  0.041773
NER_Tag_gemini-1.5-flash     0.116832  0.119794  0.146178  0.127601
NER_Tag_gemini-1.5-flash-8b  0.121869  0.116946  0.108384  0.115733
NER_Tag_gemini-1.5-pro      